# 1.Intrducció
L'spam o correu no sol·licitat, és una de les amenaces més perilloses que volten en el futur de la societat de la informació.

Aquest correu no sol·licitat és una gran eina pel món publicitari degut al baix cost i gran abast que proporcionen els serveis de missatgeria online. No obstant el creixement de l'spam és una amenaça a la convivència i a l'eficàcia dels correus electrònics, és per això que les grans distribuïdores de correus electrònics ja han realitzat algoritmes que permeten classificar aquest tipus de correu i emmagatzemar-lo a la carpeta d'spam per tal de facilitar l'ús del correu als seus usuaris i millorar la seva experiència.

![image info](https://scontent-mad1-1.xx.fbcdn.net/v/t1.6435-9/66518471_1296381087186397_6551382846613749760_n.png?_nc_cat=104&ccb=1-5&_nc_sid=9267fe&_nc_ohc=UuU7no3n1CgAX-gyEFe&_nc_ht=scontent-mad1-1.xx&oh=d12b60033a57b7aa4d62a5a66d0a26ec&oe=61D7A24D)


En aquest notebook realitzarem un anàlisi del dataset [Email Spam Classification](https://www.kaggle.com/balaka18/email-spam-classification-dataset-csv) de la plataforma [Kaggle](https://www.kaggle.com/kaggle) i intentarem classificar de manera automàtica mitjançant machine learning els correus spam que es troben dins del dataset.



# 2. Dependències

In [33]:
import pandas as pd

# 3. Data mining

### Data analysis
Realitzarem un estudi del nostre dataset, veurem el numero d'atributs que conté, el nùmero de mostres, el número de valors nuls i apart també veurem el tipus de variables dels nostres atributs i aixi podrem entendre una mica millor el nostre dataset i veure quines seran les variables predictores i quina la nostre variable objectiu.

In [34]:
df = pd.read_csv("..\data\emails.csv")
print("Nombre d'emails: ", df.shape[0])
print("Nombre d'atributs per cada email: ", df.shape[1])
print("Nombre de valors Nulls: ",df.isnull().sum().sum())

Nombre d'emails:  5172
Nombre d'atributs per cada email:  3002
Nombre de valors Nulls:  0


In [35]:
df.head()

,Email No.,the,to,ect,and,for,of,a,you,hou,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,Email 1,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Email 2,8,13,24,6,6,2,102,1,27,...,0,0,0,0,0,0,0,1,0,0
2,Email 3,0,0,1,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Email 4,0,5,22,0,5,1,51,2,10,...,0,0,0,0,0,0,0,0,0,0
4,Email 5,7,6,17,1,5,2,57,0,9,...,0,0,0,0,0,0,0,1,0,0


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5172 entries, 0 to 5171
Columns: 3002 entries, Email No. to Prediction
dtypes: int64(3001), object(1)
memory usage: 118.5+ MB


El fitxer csv conté **5172** files, on cada fila representa un correu electronic. 

Per a cada correu electrònic hi ha **3002** columnes, on la primera columna indica el nom del correu electrònic en format numèric per tal de protegir la privadesa de l'usuari, l'última columna té les etiquetes de predicció on 0 significa que no es spam i 1 significa que si és spam, la resta de **3000** columnes son el nombre de vegades que apareixen les paraules més comunes a tots els correus electrònics.

Com veiem tots els nostres atributs son valors numèrics, excepte el primer camp que es un string que com hem dit abans indica l'usuari de l'email.

També podem veure que el nostre dataset conté **0** valors nulls.

Una vegada visualitzat i analitzat el nostre dataset, podrem dir que la variable objectiu serà **"Prediction"** (l'ultim atribut de les mostres), i les variables predictores seran les columnes amb el nombre de vegades que apareix cada paraula (tots els atributs menys el primer "Email No." i l'ultim "Prediction").

### Data preparation
Realitzarem les transformacions necesaries al nostre dataset per tal de poder utilitzarlo alhora de realitzar el nostre estudi.

Com que tenim una gran quantitat de variables predictores, ens es molt complicat eliminar variables segons les correlacions, apart no tendria cap sentit fer-ho ja que cada paraula es important alhora de predir si un email es spam o no.

L'únic canvi que realitzarem serà eliminar la primera columna i posarla com a index del dataframe.

In [37]:
df = df.set_index('Email No.', drop=True)
df

,the,to,ect,and,for,of,a,you,hou,in,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
Email No.,,,,,,,,,,,,,,,,,,,,,
Email 1,0,0,1,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Email 2,8,13,24,6,6,2,102,1,27,18,...,0,0,0,0,0,0,0,1,0,0
Email 3,0,0,1,0,0,0,8,0,0,4,...,0,0,0,0,0,0,0,0,0,0
Email 4,0,5,22,0,5,1,51,2,10,1,...,0,0,0,0,0,0,0,0,0,0
Email 5,7,6,17,1,5,2,57,0,9,3,...,0,0,0,0,0,0,0,1,0,0
Email 6,4,5,1,4,2,3,45,1,0,16,...,0,0,0,0,0,0,0,0,0,1
Email 7,5,3,1,3,2,1,37,0,0,9,...,0,0,0,0,0,0,0,0,0,0
Email 8,0,2,2,3,1,2,21,6,0,2,...,0,0,0,0,0,0,0,1,0,1
Email 9,2,2,3,0,0,1,18,0,0,3,...,0,0,0,0,0,0,0,0,0,0


# 4. Model Learning

# 5. Resultats

# 6. Conclusions